# Medical RAG System Demo

This notebook demonstrates the complete medical RAG pipeline for ICD-10-CM and CPT code lookup.

## Features:
- UMLS synonym expansion
- Hybrid retrieval (Semantic + BM25)  
- Cross-encoder reranking
- Medical specialty clustering
- Hallucination detection


In [ ]:
import sys
from pathlib import Path
sys.path.append('../src')

from data_loader import MedicalDataLoader
from retrieval import HybridRetriever  
from generation import MedicalResponseGenerator
from eval import MedicalRAGEvaluator

import structlog
structlog.configure(
    processors=[structlog.dev.ConsoleRenderer()],
    logger_factory=structlog.PrintLoggerFactory(),
    cache_logger_on_first_use=True,
)

print("Medical RAG System - Demo Ready!")


Data Loading    

In [ ]:
# Initialize data loader
loader = MedicalDataLoader()

# Load sample medical codes
print("Loading ICD-10-CM codes...")
icd_codes = loader.parse_icd10_codes(Path("../data/sample.xml"))

print("Loading CPT codes...")  
cpt_codes = loader.parse_cpt_codes(Path("../data/sample.xlsx"))

# Combine and create chunks
all_codes = icd_codes + cpt_codes
chunks = loader.create_chunks_with_metadata(all_codes)

print(f"✅ Loaded {len(all_codes)} medical codes")
print(f"Sample chunk: {chunks[0]}")


Initialize Retrieval System

In [ ]:
# Initialize hybrid retriever
retriever = HybridRetriever()
retriever.initialize_vectorstore(chunks)

print("✅ Retrieval system initialized with hybrid search capabilities")
print("Features enabled:")
print("- UMLS synonym expansion")  
print("- Multi-query generation")
print("- Semantic + BM25 fusion") 
print("- Cross-encoder reranking")
print("- Medical specialty clustering")


In [ ]:
Test Retrieval

In [ ]:
# Test queries
test_queries = [
    "patient with acute chest pain and dyspnea",
    "routine office visit for diabetes management",
    "migraine headache treatment"
]

for query in test_queries:
    print(f"\n🔍 Query: {query}")
    print("-" * 50)
    
    results = retriever.retrieve(query)
    
    for i, doc in enumerate(results):
        code = doc.metadata.get('code', 'N/A')
        desc = doc.metadata.get('description', doc.page_content)
        specialty = doc.metadata.get('specialty', 'general')
        
        print(f"{i+1}. [{code}] {desc}")
        print(f"   Specialty: {specialty}")


In [ ]:
# Initialize generator
generator = MedicalResponseGenerator()

query = "patient with acute chest pain and dyspnea"
retrieved_docs = retriever.retrieve(query)

print(f"Query: {query}\n")

# Generate response
result = generator.generate_response(query, retrieved_docs)

print("Generated Response:")
print("=" * 60)
print(result['response'])
print("\n" + "=" * 60)

print(f"\nExtracted Codes: {result['codes']}")
print(f"Confidence: {result['confidence']}")
print(f"Hallucination Score: {result['hallucination_score']:.3f}")
print(f"Is Reliable: {result['is_reliable']}")
